In [32]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from tensorflow import keras


In [33]:
data = pd.read_csv("exam_data (1).csv")

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values



# ***Improving a given ANN model on a tabular dataset by:***

**We should only fit the StandardScaler on your training data to prevent data leakage.**

In [34]:
# -------------------------
# Train / validation split
# -------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
#ISSUE to be fixed

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)   # <-- FIX: Use transform

Q. Explain why it is a problem and how your fix helps

Ans. Previously it was fiting the x validation also so it was a mistake and will occure problem on our model

**Fixing a hidden mistake** AND **Changing the model architecture**

In [35]:
# -------------------------
# NEW ANN (STRONG DESIGN)
# -------------------------

#now improving
from tensorflow.keras.layers import Dropout

model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dropout(0.2), #USING DROPOUT SO IT DONT OVERFIT AND IMPROVING GENERALIZATION
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),

    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    verbose=1
)

#improved the model architecture

Epoch 1/30


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.6561 - loss: 0.6291 - val_accuracy: 0.8125 - val_loss: 0.5172
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8150 - loss: 0.4585 - val_accuracy: 0.8750 - val_loss: 0.4032
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9050 - loss: 0.3127 - val_accuracy: 0.9000 - val_loss: 0.3237
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9163 - loss: 0.2352 - val_accuracy: 0.9125 - val_loss: 0.2683
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9007 - loss: 0.2323 - val_accuracy: 0.9000 - val_loss: 0.2440
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9317 - loss: 0.1909 - val_accuracy: 0.8875 - val_loss: 0.2234
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9469 - loss: 0.1445 - val_accuracy: 0.9125 - val_loss: 0.2117
Epoch 8/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9575 - loss: 0.1454 - val_accuracy: 0.9125 - val_loss: 0.2020
Ep

Explain why you made each architectural change.

The previous architecture was not having neurons and there was only few layers and no generalization, i have added more layers with regularization

**Analyzing prediction behavior**

In [36]:
# -------------------------
# Evaluation
# -------------------------
probs = model.predict(X_val)

# 👉 Students change threshold here
# I change from 0.5 to 0.65
threshold = 0.65
preds = (probs > threshold).astype(int)


cm = confusion_matrix(y_val, preds)

print("\nFinal Training Accuracy:", round(history.history['accuracy'][-1], 4))
print("Final Validation Accuracy:", round(history.history['val_accuracy'][-1], 4))
print("Confusion Matrix:\n", cm)

# BEFORE THE TRAINING ACCURACY WAS 94% NOW IS 99%
# BEFORE THE VALIDATION ACCURACY WAS 88% NOW IS 93%

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step

Final Training Accuracy: 0.9969
Final Validation Accuracy: 0.9375
Confusion Matrix:
 [[50  1]
 [ 4 25]]


Q. Which class benefited?


Ans. The both classes benefitted as the model improved amazingly and impacting both the classes

Q. Why accuracy may not change much but confusion matrix does?

Ans. The accuracy is not dependent on confusion matrix but the confusion matrix tells us the what was the actual value of that class and what we predicted

In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, confusion_matrix, classification_report)
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# 1. Read Dataset & Ignore Date/Time
def load_and_clean_columns(filepath, columns_to_drop):

    try:
        df = pd.read_csv(filepath, sep=';', decimal=',')


        # Drop columns if they exist
        df.drop(columns=[col for col in columns_to_drop if col in df.columns], axis=1, inplace=True)
        print(f"Dataset loaded. Shape: {df.shape}")
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None


In [50]:
# 2. Preprocessing (Missing, Duplicates, Zero Sum Cols)
def preprocess_data(df):
    """
    Handles missing values, duplicates, and drops zero-sum columns.
    """
    initial_shape = df.shape

    df = df.drop_duplicates()

    df = df.dropna()

    numeric_cols = df.select_dtypes(include=[np.number]).columns
    cols_sum_zero = [col for col in numeric_cols if df[col].sum() == 0]
    df.drop(columns=cols_sum_zero, axis=1, inplace=True)

    print(f"Preprocessing complete. Rows dropped: {initial_shape[0] - df.shape[0]}. New Shape: {df.shape}")
    return df



In [49]:
# 3. Correlation & Feature Selection
def select_features_by_correlation(df, target_col, threshold=0.6):
    """
    Calculates correlation with label, filters by threshold,
    and returns selected feature names and a summary dataframe.
    """
    if target_col not in df.columns:
        raise ValueError(f"Target column '{target_col}' not found in dataframe.")

    corr_matrix = df.corr()

    target_corr = corr_matrix[target_col].drop(target_col) # Drop target itself

    target_corr_sorted = target_corr.abs().sort_values(ascending=False)

    selected_features = target_corr_sorted[target_corr_sorted > threshold].index.tolist()

    corr_df = pd.DataFrame({
        'Feature Name': selected_features,
        'Correlation Score': target_corr_sorted[selected_features].values
    })

    print(f"\n--- Feature Selection (Threshold > {threshold}) ---")
    print(corr_df)

    return selected_features, corr_df


In [48]:

# 4. Standard Scaler
def scale_features(X_data):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_data)
    return X_scaled

In [47]:
def train_and_evaluate(model, model_name, params_desc, X_train, X_test, y_train, y_test):

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    print(f"\nResults for: {model_name} | Params: {params_desc}")
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")

    print("Classification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))

    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix: {model_name}')
    plt.show()

    return {'Model': model_name, 'Params': params_desc, 'Accuracy': acc, 'F1': f1}

In [46]:

# --- CONFIGURATION ---
dataset_path = 'df_kwh_adjusted.csv'

target_col_name = 'label'

cols_to_ignore = ['date', 'time', 'day', 'Unnamed: 0']

# 1. Read Data
df = load_and_clean_columns(dataset_path, cols_to_ignore)

# 2. Preprocessing
df = preprocess_data(df)

if target_col_name not in df.columns:
    print(f"CRITICAL ERROR: Column '{target_col_name}' not found. Please update 'target_col_name' in main().")
    # For demonstration purposes, I will create a dummy target if it's missing so code runs
    print("Creating dummy target for demonstration...")
    df[target_col_name] = np.random.randint(0, 2, df.shape[0])

# 3. Feature Selection
selected_feats, corr_info = select_features_by_correlation(df, target_col_name, threshold=0.01)


if len(selected_feats) == 0:
    print("No features met the threshold. Lower the threshold or check data.")

X = df[selected_feats]
y = df[target_col_name]


X_scaled = scale_features(X)

X_unscaled = X.values

feature_sets = {
    "Set1 (Scaled)": X_scaled,
    "Set2 (Unscaled)": X_unscaled
}


models_config = [
    # Naive Bayes (GaussianNB doesn't have many hyperparameters to tune, usually just var_smoothing)
    ("Naive Bayes", GaussianNB(), [{"var_smoothing": 1e-9}, {"var_smoothing": 1e-8}, {"var_smoothing": 1e-7}, {"var_smoothing": 1e-6}, {"var_smoothing": 1e-5}]),

    # Decision Tree (Criteria, MaxDepth)
    ("Decision Tree", DecisionTreeClassifier(), [
        {"criterion": "gini", "max_depth": 3},
        {"criterion": "entropy", "max_depth": 5},
        {"criterion": "gini", "max_depth": 10},
        {"criterion": "entropy", "max_depth": None},
        {"splitter": "random", "max_depth": 5}
    ]),

    # Random Forest (N_estimators, MaxDepth)
    ("Random Forest", RandomForestClassifier(), [
        {"n_estimators": 10, "max_depth": 5},
        {"n_estimators": 50, "max_depth": 10},
        {"n_estimators": 100, "criterion": "entropy"},
        {"n_estimators": 20, "max_features": "log2"},
        {"n_estimators": 50, "max_depth": None}
    ]),

    # KNN (n_neighbors, weights)
    ("KNN", KNeighborsClassifier(), [
        {"n_neighbors": 3},
        {"n_neighbors": 5},
        {"n_neighbors": 7, "weights": "distance"},
        {"n_neighbors": 9},
        {"n_neighbors": 5, "metric": "manhattan"}
    ]),

    # Logistic Regression (C, penalty)
    ("Logistic Regression", LogisticRegression(max_iter=1000), [
        {"C": 1.0, "solver": "lbfgs"},
        {"C": 0.1, "solver": "lbfgs"},
        {"C": 10.0, "solver": "liblinear"},
        {"C": 0.01, "solver": "liblinear"},
        {"C": 5.0, "solver": "lbfgs"}
    ])
]

best_outcome = {'Score': 0, 'Info': ''}

for set_name, X_data in feature_sets.items():
    print(f"\n{'='*20} Processing {set_name} {'='*20}")

    X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2, random_state=42)

    for name, model_obj, param_list in models_config:

        # Run 5 iterations with different parameters
        for params in param_list:
            # Set parameters
            model_obj.set_params(**params)

            # Train and Evaluate
            res = train_and_evaluate(model_obj, name, str(params), X_train, X_test, y_train, y_test)

            # Check best outcome
            if res['Accuracy'] > best_outcome['Score']:
                best_outcome['Score'] = res['Accuracy']
                best_outcome['Info'] = f"{name} with {params} on {set_name}"

print(f"BEST OUTCOME: {best_outcome['Info']} with Accuracy: {best_outcome['Score']:.4f}")

Dataset loaded. Shape: (8354, 370)
Preprocessing complete. Rows dropped: 1734. New Shape: (6620, 370)
CRITICAL ERROR: Column 'label' not found. Please update 'target_col_name' in main().
Creating dummy target for demonstration...

--- Feature Selection (Threshold > 0.01) ---
   Feature Name  Correlation Score
0        MT_290           0.030125
1        MT_314           0.025638
2        MT_231           0.025187
3        MT_211           0.022547
4        MT_294           0.020792
5        MT_207           0.019138
6        MT_245           0.018847
7        MT_257           0.016989
8        MT_219           0.016981
9        MT_218           0.016046
10       MT_220           0.015788
11       MT_261           0.015736
12       MT_208           0.014683
13       MT_317           0.014650
14       MT_217           0.013145
15       MT_223           0.012679
16       MT_246           0.012577
17       MT_215           0.012452
18       MT_244           0.012138
19       MT_210         

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Results for: Logistic Regression | Params: {'C': 1.0, 'solver': 'lbfgs'}
Accuracy: 0.5136, Precision: 0.5123, Recall: 0.5136, F1: 0.5071
Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.63      0.57       674
           1       0.51      0.40      0.44       650

    accuracy                           0.51      1324
   macro avg       0.51      0.51      0.51      1324
weighted avg       0.51      0.51      0.51      1324



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Results for: Logistic Regression | Params: {'C': 0.1, 'solver': 'lbfgs'}
Accuracy: 0.5098, Precision: 0.5083, Recall: 0.5098, F1: 0.5032
Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.62      0.56       674
           1       0.50      0.39      0.44       650

    accuracy                           0.51      1324
   macro avg       0.51      0.51      0.50      1324
weighted avg       0.51      0.51      0.50      1324


Results for: Logistic Regression | Params: {'C': 10.0, 'solver': 'liblinear'}
Accuracy: 0.5113, Precision: 0.5099, Recall: 0.5113, F1: 0.5046
Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.63      0.57       674
           1       0.50      0.39      0.44       650

    accuracy                           0.51      1324
   macro avg       0.51      0.51      0.50      1324
weighted avg       0.51      0.51      0.50      1324


Results for: Logis

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
